In [1]:
%reload_ext autoreload
%autoreload 2



In [2]:
#クリーニングしたwebテキストをクラスタリングするモデルを作る
n_clusters=100

In [3]:
from src.cleaner.auto_cleaner import clean_text
from datasets import load_dataset

/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
txt="こんにちは､げんきですか?研究を "
clean_text(txt)

'こんにちは、げんきですか?研究を'

In [6]:
#gzファイルの一覧を取得
import glob
import os
with open("temp/gz_list.txt","r") as f:
    gz_list=f.readlines()

gz_list=[i for i in gz_list if i!="\n"]
gz_list=[i.replace("\n","") for i in gz_list]
gz_list

['../data/original_dump/oscar/oscar_ja_0.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_1.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_2.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_3.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_4.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_5.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_6.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_7.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_8.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_9.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_10.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_11.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_12.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_13.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_14.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_15.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_16.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_17.jsonl.gz',
 '../data/original_dump/oscar/oscar_ja_18.jsonl.gz',
 '.

In [11]:
from src.load_gz import read_gzip_json_file
lines=[]
path=gz_list[0]
for article in read_gzip_json_file(path):
    text = article.get('text', '')  # 'text'キーからテキストデータを取得
    lines.append(text)

In [12]:
lines[:3]

['結果: 表示 1 a 25 de 88 結果\nBadalona Municipal Museum. Badalona. Arts and culture\n… Plaza de L\'Assemblea de Catalunya 1. 08911. Badalona. Barcelona. (Catalonia) E-mail: info@museudebadalona.cat. http://www.museudebadalona.cat/. A Roman past. This museum recalls the town\'s Roman heritage through its public baths, sculptures and other objects. Badalona, once called Baetulo, was found… Triumphal Arch. Barcelona. Arts and culture\n… Paseo Lluis Companys s/n. 08010. Barcelona. (Catalonia) It was the entry gate to the International Exposition of 1888. The monument is an allegory to industry, handicrafts and business, and was designed by the architect Josep Vilaseca. The Triumphal Arch was designed by the architect Josep Vilaseca … Horta Maze. Barcelona. Arts and culture\n… Calle Germans Desvalls (Parc del Laberint ) s/n. 08035. Barcelona. (Catalonia) Fax.: +34 934132424 | E-mail: parcsijardins@mail.bcn.es. www.bcn.es/parcsijardins. It was created when a Catalan aristocrat decided to convert

In [13]:
import random
random.shuffle(gz_list)

#クラスタリングに使うデータベースの数
n_gz=100

train_datasets=gz_list[:n_gz]

In [17]:
from tqdm import tqdm

#各データセットごと､N件のデータを取得
max_articles=5000
cleaned_text=[]

for path in tqdm(train_datasets):
    lines=[]
    for article in read_gzip_json_file(path):
        text = article.get('text', '')  # 'text'キーからテキストデータを取得
        lines.append(text)

    cnt=0
    for text in (lines):
        #text=next(dataset_iter)["text"]
        text=clean_text(text)

        if text!="":
            cleaned_text.append(text)
            cnt+=1
            if cnt>=max_articles:
                break


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [13:08<00:00, 39.41s/it]


# クラス分類

In [18]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec,texts2classes
import numpy as np


In [19]:

t2v=Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

In [20]:
title_vecs=[t2v.text2vec(i) for i in tqdm(cleaned_text)]

  0%|          | 92/100000 [00:00<03:43, 447.60it/s]

100%|██████████| 100000/100000 [04:21<00:00, 382.95it/s]


In [21]:
title_vecs=np.array(title_vecs)
# k-meansクラスタリング
print("clustering...")
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=1).fit(title_vecs)


# 各データポイントが割り当てられたクラスタのインデックスを取得
labels = kmeans.labels_

# 各クラスタに含まれるデータポイントの数を計算
cluster_counts = dict((i, list(labels).count(i)) for i in range(n_clusters))

print(cluster_counts)

clustering...
{0: 1291, 1: 1832, 2: 1067, 3: 457, 4: 857, 5: 1257, 6: 1226, 7: 1691, 8: 421, 9: 1193, 10: 904, 11: 688, 12: 2231, 13: 955, 14: 2145, 15: 533, 16: 342, 17: 1675, 18: 1180, 19: 403, 20: 191, 21: 617, 22: 218, 23: 951, 24: 1637, 25: 790, 26: 425, 27: 1590, 28: 726, 29: 676, 30: 2265, 31: 940, 32: 745, 33: 1667, 34: 761, 35: 1643, 36: 1129, 37: 993, 38: 541, 39: 1670, 40: 428, 41: 1040, 42: 2215, 43: 1121, 44: 463, 45: 785, 46: 639, 47: 799, 48: 867, 49: 643, 50: 1109, 51: 1855, 52: 1065, 53: 777, 54: 1828, 55: 1391, 56: 496, 57: 433, 58: 2151, 59: 720, 60: 631, 61: 626, 62: 2088, 63: 845, 64: 471, 65: 1024, 66: 1194, 67: 1295, 68: 402, 69: 964, 70: 915, 71: 1137, 72: 491, 73: 1131, 74: 366, 75: 2303, 76: 1255, 77: 740, 78: 367, 79: 343, 80: 375, 81: 1092, 82: 429, 83: 403, 84: 330, 85: 2772, 86: 492, 87: 203, 88: 971, 89: 1605, 90: 223, 91: 2223, 92: 534, 93: 870, 94: 2187, 95: 598, 96: 765, 97: 1008, 98: 538, 99: 446}


In [22]:
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances
# クラスタのセントロイドを取得
# 仮に title_vecs がデータセットの特徴ベクトル、cleaned_text がそれに対応するクリーンなテキストデータを表すとします。
# n_clusters には、生成したいクラスタの数を指定します。

centroids = kmeans.cluster_centers_

# 各クラスタに属するデータポイントのインデックスを取得
cluster_indices = {i: np.where(kmeans.labels_ == i)[0] for i in range(n_clusters)}

for cluster_id, indices in cluster_indices.items():
    # クラスタ内のデータポイントとセントロイドとの距離を計算
    distances = pairwise_distances(title_vecs[indices], [centroids[cluster_id]])
    # 距離が最も小さい上位5つのデータポイントを選択
    closest_indices = np.argsort(distances[:, 0])[:5]
    
    print(f"Cluster {cluster_id}:")
    for idx in closest_indices:
        # クラスタ内で選択されたデータポイントの実際のインデックス
        actual_idx = indices[idx]
        # テキストが長い場合は100文字に制限し、改行を削除
        print(f"  - {cleaned_text[actual_idx].replace('\n', '')[:100]}")
    print("\n")

Cluster 0:
  - 概要スマートな4つ星のクラリオンホテルヨーロッパは、Linda Monument、Kadriorg ParkとNational Library of Estoniaから数分のビーチにあります。2007
  - 元となる情報1288レビュー概要当モダンなヴィスコンティパレスホテルはフォロ・ロマーノ、Vittorio Emmanuel II Monumentとコロッセウムから数分のところの重要な場所にあります。
  - 前回は開業直後に訪れたが、あれから6年半・・・木々が大きくなり、より深くなった熱帯雨林の中に、日々お客様にご利用いただき従業員によって磨かれたラグジュアリーな施設が、しっくりと馴染み優雅な佇まいを見せ
  - 【2020年版】ダナンでスパの口コミが良いホテルランキング7選!時間や料金もご紹介|海外旅行・海外ツアーはトラベル・スタンダード・ジャパン30種のスパ受け放題!全室プライベートプール付きのヴィラで「プ
  - アメリカ合衆国のホテルカリフォルニアのホテルロサンゼルスのホテルヒルトンロサンゼルスエアポートアンドタワーズ(Hilton Los Angeles Airport)最低価格¥9,230"ヒルトンロサン


Cluster 1:
  - コンピュータ|自由で気ままな日々シーズン2カテゴリー:コンピュータ|コメントをどうぞ「グーグル左うざい」なので、ユーザスタイルシートで移動させました。投稿日時:2010年5月7日投稿者:twleewa
  - 6月,2003の投稿を表示していますMacOS Xにはいとーけーさん作の「CloCal Pro」というデスクトップに時計とカレンダーを表示するフリーソフトがあります。このカレンダーにiCalの予定表と
  - 新しいウェブメールについて情報交換するスレッドこちら修正入りました。モバイル表示の切り替えが出来ない。Vivaldiと相性の悪い拡張まとめスレッド。Googleの拡張機能が使えない問題について。Goo
  - 端末への一時的なデータ保管先として、ApplicationのState領域があります。Mangoには日本語フォント(Yu GothicやMeiryo UI等)が含まれているため、日本語の表示に関しては
  - 2009年の記事一覧|bit part合同会社MT5の

In [25]:
import joblib
#save model
joblib.dump(kmeans, "../data/model/kmeans.pkl")

['../data/model/kmeans.pkl']

In [26]:
target_texts=cleaned_text[:10]


print(texts2classes(target_texts,t2v,kmeans))

[78 15 12 85  5 49 69 88 60 55]
